In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import datetime

In [6]:
import pandas as pd

# Cell 1: Load Data


try:
    df_training = pd.read_csv('Final_data (1).csv', encoding='utf-8', low_memory=False)
    df_fraud_claims1 = pd.read_csv('MMC Fraud Cases 2_7_2025.csv', encoding='utf-8', low_memory=False)
    df_fraud_claims2 = pd.read_csv('RISK REGISTER- 2n July 2025 Updated.csv', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    print("UTF-8 decoding failed. Trying 'latin-1' encoding...")
    df_training = pd.read_csv('Final_data (1).csv', encoding='latin-1', low_memory=False)
    df_fraud_claims1 = pd.read_csv('MMC Fraud Cases 2_7_2025.csv', encoding='latin-1', low_memory=False)
    df_fraud_claims2 = pd.read_csv('RISK REGISTER- 2n July 2025 Updated.csv', encoding='latin-1', low_memory=False)
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure the file paths are correct and the files are accessible.")


print("Training Data Head:")
print(df_training.head())
print("\nFraud Claims 1 Head:")
print(df_fraud_claims1.head())
print("\nFraud Claims 2 Head:")
print(df_fraud_claims2.head())

An error occurred: [Errno 2] No such file or directory: 'Final_data (1).csv'
Please ensure the file paths are correct and the files are accessible.
Training Data Head:


NameError: name 'df_training' is not defined

In [ ]:
# Identify columns with low cardinality (few unique values)
categorical_cols = [
    'relationship',
    'gender(claimant)',
    'company',
    'broad_benefit',
    'benefit',
    'main_member_gender'
]

for col in categorical_cols:
    if col in df_training.columns:
        df_training[col] = df_training[col].astype('category')

# Optional: Check 'provider' and 'ailments' as well
# Check number of unique values first to ensure it's not too high
if 'provider' in df_training.columns and df_training['provider'].nunique() < 5000:
    df_training['provider'] = df_training['provider'].astype('category')

if 'ailments' in df_training.columns and df_training['ailments'].nunique() < 10000:
    df_training['ailments'] = df_training['ailments'].astype('category')

# You can check the memory usage before and after to see the improvement
print("Data types after optimization:")
print(df_training.info())

In [ ]:
# Cell 2: Initial Data Cleaning and Preprocessing (Training Data)


df_training.columns = df_training.columns.str.strip().str.lower().str.replace(' ', '_')
df_fraud_claims1.columns = df_fraud_claims1.columns.str.strip().str.lower().str.replace(' ', '_')
df_fraud_claims2.columns = df_fraud_claims2.columns.str.strip().str.lower().str.replace(' ', '_')

# Standardize column names where possible across datasets for potential merging or lookups
df_fraud_claims1 = df_fraud_claims1.rename(columns={'claim_amount': 'total_payable', 'hospital': 'payee'})

# Convert 'ailment_date' and 'dob' to datetime objects in df_training
# Specify the format '%d/%m/%Y' to correctly parse Day/Month/Year dates
df_training['ailment_date'] = pd.to_datetime(df_training['ailment_date'], format='%d/%m/%Y', errors='coerce')
#df_training['dob'] = pd.to_datetime(df_training['dob'], format='%d/%m/%Y', errors='coerce')
df_training['date_of_birth(claimant)'] =  pd.to_datetime(df_training['date_of_birth(claimant)'], format='%d/%m/%Y', errors='coerce')
# This is the crucial part that creates the 'age' column.
current_year = pd.Timestamp.now().year
df_training['age'] = current_year - df_training['date_of_birth(claimant)'].dt.year # Ensure this line is run!


# Clean 'totals' and 'limit_amount' columns in df_training
df_training['totals'] = pd.to_numeric(df_training['total_payable'], errors='coerce')
df_training['limit_amount'] = pd.to_numeric(df_training['cover_limit'], errors='coerce')

# Display info to check data types after conversion
print("\nTraining Data Info after date conversion:")
print(df_training.info())

In [ ]:
# Cell 3: Feature Engineering: Days Since Last Visit
# Comments: Calculate the number of days since the last visit for each claimant.
# This feature helps identify unusual claim frequency.

df_training = df_training.sort_values(by=['claimant_suddo', 'ailment_date'])
df_training['days_since_last_visit'] = df_training.groupby('claimant_suddo')['ailment_date'].diff().dt.days.fillna(0)

print("\nDays Since Last Visit Feature:")
print(df_training[['claimant_suddo', 'ailment_date', 'days_since_last_visit']].head())


In [ ]:
# Cell 4: Feature Engineering: Grouping Features for Broad Benefit Categories (6 Categories)
# Comments: Create binary flags for six specific broad benefit categories: IN-PATIENT, OUT-PATIENT, OPTICAL, DENTAL, MATERNITY, LAST EXPENSE.
# This categorizes claims for specific analysis based on these distinct types.

# Standardize the 'broad_benefit' column to lower case for consistent matching
df_training['broad_benefits_cleaned'] = df_training['broad_benefit'].astype(str).str.lower().str.strip()

# Create individual flags for each of the 6 broad benefit categories
# Using .str.contains() for flexibility, as one broad benefit entry might cover multiple types (e.g., "Inpatient Dental")
df_training['is_inpatient_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('in-patient|inpatients', na=False)).astype(int)
df_training['is_outpatient_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('out-patient|outpatients', na=False)).astype(int)
df_training['is_optical_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('optical', na=False)).astype(int)
df_training['is_dental_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('dental', na=False)).astype(int)
df_training['is_maternity_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('maternity', na=False)).astype(int)
df_training['is_last_expense_benefit'] = (df_training['broad_benefits_cleaned'].str.contains('last expense', na=False)).astype(int)

# Note: A single 'broad_benefit' value might contain multiple keywords (e.g., 'inpatient optical').
# The .str.contains() method will flag if *any* of the keywords are present.
# If strict, mutually exclusive categories are needed (e.g., a claim can only be *either* inpatient *or* optical, but not both),
# then this logic would need to be re-evaluated to enforce exclusivity.

print("\nBroad Benefit Category Flags:")
print(df_training[['broad_benefit', 'is_inpatient_benefit', 'is_outpatient_benefit',
                   'is_optical_benefit', 'is_dental_benefit', 'is_maternity_benefit',
                   'is_last_expense_benefit']].head())

In [ ]:
# Cell 5: Feature Engineering: Calculate Means, Deviations, and Variances for Claims
# Comments: Calculate mean, standard deviation, and variance of 'totals' (claim amounts) per claimant.
# These statistical measures help identify claims significantly different from a claimant's average.

df_training['claim_amount_mean_by_claimant'] = df_training.groupby('claimant_suddo')['totals'].transform('mean')
df_training['claim_amount_dev_by_claimant'] = df_training.groupby('claimant_suddo')['totals'].transform('std').fillna(0)
df_training['claim_amount_var_by_claimant'] = df_training.groupby('claimant_suddo')['totals'].transform('var').fillna(0)

print("\nClaim Amount Statistics by Claimant:")
print(df_training[['claimant_suddo', 'totals', 'claim_amount_mean_by_claimant', 'claim_amount_dev_by_claimant', 'claim_amount_var_by_claimant']].head())


In [ ]:
# Cell 6: Feature Engineering: Average per Broad Benefit and Deviation
# Comments: Calculate the average claim amount and its deviation for each broad benefit type.
# This helps in identifying claims that are outliers within their specific broad benefit category.

df_training['average_claim_per_broad_benefit'] = df_training.groupby('broad_benefits_cleaned')['totals'].transform('mean')
df_training['deviation_claim_per_broad_benefit'] = df_training.groupby('broad_benefits_cleaned')['totals'].transform('std').fillna(0)

print("\nClaim Statistics by Broad Benefit Type (re-calculated using cleaned column from Cell 4):")
print(df_training[['broad_benefit', 'totals', 'average_claim_per_broad_benefit', 'deviation_claim_per_broad_benefit']].head())



In [ ]:
# Cell 7: Feature Engineering: Age and Status (Child/Spouse) Mismatch
# Comments: Calculate age from DOB and identify potential mismatches based on 'REL' (relationship)
# and main member's gender and age.

# 'age' column (claimant's age) should already be calculated in Cell 2.

# Simple check for 'CHILD' relationship: if relationship is 'CHILD' but age is > 23
df_training['age_rel_mismatch_flag'] = ((df_training['relationship'].astype(str).str.upper() == 'CHILD') & (df_training['age'] > 25)).astype(int)

# Initialize flags to 0 in case the relevant columns are missing
df_training['spouse_gender_mismatch_flag'] = 0
df_training['spouse_age_mismatch_flag'] = 0

# --- Gender mismatch for spouses ---
# Check if the 'main_member_gender' column exists in the DataFrame
if 'main_member_gender' in df_training.columns:
    # Convert claimant's gender and main member's gender to uppercase for robust comparison
    df_training['gender(claimant)_upper'] = df_training['gender(claimant)'].astype(str).str.upper()
    df_training['main_member_gender_upper'] = df_training['main_member_gender'].astype(str).str.upper()

    # Flag if 'REL' is 'SPOUSE' AND claimant's gender is the same as the main member's gender.
    # (Assuming a mismatch means different genders for a spouse pair)
    df_training['spouse_gender_mismatch_flag'] = (
        (df_training['relationship'].astype(str).str.upper() == 'SPOUSE') &
        (df_training['gender(claimant)_upper'] == df_training['main_member_gender_upper'])
    ).astype(int)

    # Clean up temporary uppercase columns
    df_training = df_training.drop(columns=['gender(claimant)_upper', 'main_member_gender_upper'])
else:
    print("Warning: 'main_member_gender' column not found after cleaning. Spouse gender mismatch flag will default to 0.")

# --- Age mismatch for spouses ---
# Check if the 'age_main_member' column exists in the DataFrame
if 'age_main_member' in df_training.columns:
    # Convert 'age_main_member' to numeric, coercing any errors to NaN
    df_training['age_main_member'] = pd.to_numeric(df_training['age_main_member'], errors='coerce')

    # Define a reasonable age difference threshold (e.g., 15 years) for a potential mismatch
    age_diff_threshold = 15

    # Flag if 'REL' is 'SPOUSE' AND:
    # 1. The absolute age difference between claimant and main member is greater than the threshold, OR
    # 2. The claimant's age (representing the spouse) is suspiciously low (e.g., under 18 years old).
    df_training['spouse_age_mismatch_flag'] = (
        (df_training['relationship'].astype(str).str.upper() == 'SPOUSE') &
        (
            (np.abs(df_training['age'] - df_training['age_main_member']) > age_diff_threshold) |
            (df_training['age'] < 18)
        )
    ).astype(int)
else:
    print("Warning: 'age_main_member' column not found after cleaning. Spouse age mismatch flag will default to 0.")


# Display results
print("\nAge and Relationship Mismatch Flags:")
# Dynamically build the list of columns to display to avoid errors if some new columns are not found
display_cols = ['claimant_suddo', 'relationship', 'gender(claimant)', 'age', 'age_rel_mismatch_flag']
if 'main_member_gender' in df_training.columns:
    display_cols.append('main_member_gender')
if 'spouse_gender_mismatch_flag' in df_training.columns:
    display_cols.append('spouse_gender_mismatch_flag')
if 'age_main_member' in df_training.columns:
    display_cols.append('age_main_member')
if 'spouse_age_mismatch_flag' in df_training.columns:
    display_cols.append('spouse_age_mismatch_flag')

print(df_training[display_cols].head())

In [ ]:
# Cell 8: Feature Engineering: Mean, Deviation, and % Deviation for Service Offered and Price Charged
# Comments: Assuming 'ailment' is the service offered and 'totals' is the price charged.
# This helps detect if a particular service is consistently over/under charged compared to its average.

df_training['service_charge_mean_by_ailment'] = df_training.groupby('ailments')['total_payable'].transform('mean')
df_training['service_charge_dev_by_ailment'] = df_training.groupby('ailments')['total_payable'].transform('std').fillna(0)
df_training['service_charge_pct_dev_by_ailment'] = (
    (df_training['total_payable'] - df_training['service_charge_mean_by_ailment']) / df_training['service_charge_mean_by_ailment']
).fillna(0)

print("\nService Charge Statistics by Ailment:")
print(df_training[['ailments', 'total_payable', 'service_charge_mean_by_ailment', 'service_charge_dev_by_ailment', 'service_charge_pct_dev_by_ailment']].head())


In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher  # Built-in fuzzy matching alternative

# Define conditions with severity weights (1-5 scale, 5 = most severe mismatch)
condition_sets = {
    # Format: (condition_patterns, gender/age_filter, weight, is_borderline)
    'female_conditions': (
        ['pregnancy', 'prenatal', 'postpartum', 'gestational', 'ovarian', 
         'cervical', 'endometriosis', 'uterine fibroids', 'menopause', 
         'pcos', 'ectopic pregnancy', 'premenstrual', 'vaginitis', 
         'pelvic inflammatory', 'mammogram', 'hysterectomy', 
         'tubal ligation', 'preeclampsia', 'placenta previa', 'uterine', 
          'vaginosis', 'vaginitis', 'abortion', 'cyesis', 'a.n.c', 'vertex delivery'

 


],
        lambda df: df['gender(claimant)'].str.upper() == 'M',
        5,  # Very severe mismatch
        False
    ),
    
    'borderline_female_conditions': (
        ['breast cancer', 'mastectomy', 'gynecologic'],
        lambda df: df['gender(claimant)'].str.upper() == 'M',
        3,  # Less severe (males can rarely have these)
        True
    ),
    
    'male_conditions': (
        ['prostate', 'testicular', 'erectile', 
         'bph', 'vasectomy', 'peyronie', 'circumcision', 'ejaculation', 'prostatic', 'sperm' ],
        lambda df: df['gender(claimant)'].str.upper() == 'F',
        5,
        False
    ),
    
    'borderline_male_conditions': (
        ['low testosterone', 'male pattern baldness'],
        lambda df: df['gender(claimant)'].str.upper() == 'F',
        2,  # Females can have testosterone issues too
        True
    ),
    
    'pediatric_conditions': (
        ['pediatric', 'well-child', 'vaccination', 'dtap', 'mmr', 'croup',
         'diaper rash', 'neonatal', 'childhood leukemia', 'adhd diagnosis',
         'pediatric epilepsy', 'congenital heart', 'school physical'],
        lambda df: df['age(claimant)'] >= 18,
        4,
        False
    ),
    
    'adult_conditions_young': (
        ['alzheimer', 'dementia', 'osteoporosis', 'colonoscopy',
         'prostate exam', 'mammogram', 'presbyopia'],
        lambda df: df['age(claimant)'] < 40,  # More granular age bracket
        4,
        False
    ),
    
    'adult_conditions_elderly': (
        ['geriatric', 'senior', 'memory care', 'assisted living'],
        lambda df: df['age(claimant)'] < 65,
        5,
        False
    ),
    
    'borderline_adult_conditions': (
        ['hypertension', 'type 2 diabetes', 'cataract'],
        lambda df: df['age(claimant)'] < 18,  # Rare but possible in children
        2,
        True
    )
}

def similar(a, b, threshold=0.85):
    """Check if strings are similar using difflib"""
    return SequenceMatcher(None, a, b).ratio() >= threshold

def detect_mismatches(df, condition_sets):
    """
    Enhanced mismatch detection without external dependencies
    Returns DataFrame with weighted mismatch scores and flags
    """
    # Initialize columns
    df['mismatch_score'] = 0
    df['definite_mismatch'] = 0
    df['borderline_mismatch'] = 0
    df['mismatch_details'] = ''
    
    # Convert ailments to lowercase for case-insensitive matching
    ailments = df['ailments'].astype(str).str.lower()
    
    # Check diagnosis name patterns
    for name, (patterns, filter_fn, weight, is_borderline) in condition_sets.items():
        # Fuzzy match using difflib
        matches = np.zeros(len(df), dtype=bool)
        for pattern in patterns:
            matches |= ailments.apply(lambda x: any(
                similar(pattern, word) 
                for word in x.split()
            ))
        
        mask = matches & filter_fn(df)
        df.loc[mask, 'mismatch_score'] += weight
        if is_borderline:
            df.loc[mask, 'borderline_mismatch'] = 1
            df.loc[mask, 'mismatch_details'] += f'[Borderline:{name}]'
        else:
            df.loc[mask, 'definite_mismatch'] = 1
            df.loc[mask, 'mismatch_details'] += f'[Definite:{name}]'
    
    # Age-specific enhancements
    df['age_bracket'] = pd.cut(
        df['age(claimant)'],
        bins=[0, 2, 12, 18, 40, 65, 120],
        labels=['infant', 'child', 'teen', 'young_adult', 'adult', 'elderly']
    )
    
    return df

# Apply to your dataframe
df_training = detect_mismatches(df_training, condition_sets)

# Create final flags based on thresholds
df_training['high_confidence_fraud'] = (
    (df_training['definite_mismatch'] == 1) & 
    (df_training['mismatch_score'] >= 4)
).astype(int)

df_training['needs_review'] = (
    (df_training['borderline_mismatch'] == 1) |
    ((df_training['mismatch_score'] >= 2) & 
     (df_training['mismatch_score'] < 4))
).astype(int)

print("\nEnhanced Mismatch Detection Results:")
print(df_training[['gender(claimant)', 'age(claimant)', 'age_bracket', 'ailments', 
                   'mismatch_score', 'high_confidence_fraud', 
                   'needs_review', 'mismatch_details']].head(10))

In [ ]:
# Cell 11: Feature Engineering: Average, Deviation, and % Deviation of Provider and Service
# Comments: Calculate statistics for 'PAYEE' (provider) and 'AILMENT' (service).
# This helps identify providers with unusually high/low charges for specific services.

df_training['provider_service_mean_charge'] = df_training.groupby(['provider', 'ailments'])['total_payable'].transform('mean').fillna(0)
df_training['provider_service_dev_charge'] = df_training.groupby(['provider', 'ailments'])['total_payable'].transform('std').fillna(0)
df_training['provider_service_pct_dev_charge'] = (
    (df_training['total_payable'] - df_training['provider_service_mean_charge']) / df_training['provider_service_mean_charge']
).fillna(0)

print("\nProvider and Service Charge Statistics:")
print(df_training[['provider', 'ailments', 'total_payable', 'provider_service_mean_charge', 'provider_service_dev_charge', 'provider_service_pct_dev_charge']].head())


In [ ]:
# Cell 12: Feature Engineering: Number of Repeat Claimants
df_training['repeat_claimant_count'] = df_training.groupby('claimant_suddo')['member_suddo'].transform('count')

print("\nRepeat Claimant Count:")
print(df_training[['claimant_suddo', 'repeat_claimant_count']].head())



In [ ]:
# Cell 13: Feature Engineering: Claim Ratio and High Claim Flag
df_training['claim_ratio'] = (df_training['total_payable'] / df_training['cover_limit']).fillna(0)
df_training['high_claim_flag'] = (df_training['total_payable'] > (0.8 * df_training['cover_limit'])).astype(int)

print("\nClaim Ratio and High Claim Flag:")
print(df_training[['total_payable', 'cover_limit', 'claim_ratio', 'high_claim_flag']].head())

In [ ]:
# Cell 14: Feature Engineering: Average Claim per User
df_training['average_claim_per_user'] = df_training.groupby('member_suddo')['total_payable'].transform('mean').fillna(0)

print("\nAverage Claim per User:")
print(df_training[['member_suddo', 'total_payable', 'average_claim_per_user']].head())



In [ ]:
# Cell 15: Feature Engineering: Claim Amount Variance (Claimant and Company)
df_training['claim_amount_variance_claimant'] = df_training.groupby('claimant_suddo')['total_payable'].transform('var').fillna(0)
df_training['claim_amount_variance_company'] = df_training.groupby('company')['total_payable'].transform('var').fillna(0)

print("\nClaim Amount Variance by Claimant and Company:")
print(df_training[['claimant_suddo', 'company', 'total_payable', 'claim_amount_variance_claimant', 
                   'claim_amount_variance_company']].head())


In [ ]:
# Cell 16: Feature Engineering: Company Total Claims
df_training['company_total_claims'] = df_training.groupby('company')['total_payable'].transform('sum').fillna(0)

print("\nCompany Total Claims:")
print(df_training[['company', 'total_payable', 'company_total_claims']].head())

In [ ]:
# Cell 17: Feature Engineering: Medical & Benefit Features
# Using the exact column names from your dataset

# Label Encoding for categorical features
le_ailment = LabelEncoder()
df_training['ailment_type_encoded'] = le_ailment.fit_transform(df_training['ailments'].astype(str))

le_benefit = LabelEncoder()
df_training['benefit_type_encoded'] = le_benefit.fit_transform(df_training['benefit'].astype(str))

le_broad_benefits = LabelEncoder()
df_training['broad_benefits_encoded'] = le_broad_benefits.fit_transform(df_training['broad_benefit'].astype(str))

# Diagnosis Group (Placeholder)
def map_diagnosis_group(ailment):
    if pd.isna(ailment):
        return 'Unknown_Diagnosis'
    ailment = str(ailment).lower()
    if 'fever' in ailment:
        return 'Fever_Related'
    elif 'pain' in ailment:
        return 'Pain_Related'
    elif 'dental' in ailment:
        return 'Dental_Related'
    elif 'maternity' in ailment or 'cysis' in ailment:
        return 'Maternity_Related'
    else:
        return 'Other_Diagnosis'

df_training['diagnosis_group'] = df_training['ailments'].apply(map_diagnosis_group)
le_diagnosis_group = LabelEncoder()
df_training['diagnosis_group_encoded'] = le_diagnosis_group.fit_transform(df_training['diagnosis_group'])

# Hospital Risk Score (from fraud claims mapping)
# Using 'payee' from fraud_claims1 and 'reported_fraud_cases' from fraud_claims2
all_fraud_hospitals_names_1 = df_fraud_claims1['payee'].dropna().astype(str).str.strip().str.lower().str.replace(' ', '_').unique()
all_fraud_hospitals_names_2 = df_fraud_claims2['reported_fraud_cases'].dropna().astype(str).str.strip().str.lower().str.replace(' ', '_').unique()
all_fraud_hospitals = np.unique(np.concatenate([all_fraud_hospitals_names_1, all_fraud_hospitals_names_2]))

df_training['provider_cleaned'] = df_training['provider'].astype(str).str.strip().str.lower().str.replace(' ', '_')

hospital_fraud_counts = {}
for hospital in all_fraud_hospitals:
    count = 0
    count += df_fraud_claims1['payee'].astype(str).str.lower().str.replace(' ', '_').fillna('').str.contains(hospital, regex=False).sum()
    count += df_fraud_claims2['reported_fraud_cases'].astype(str).str.lower().str.replace(' ', '_').fillna('').str.contains(hospital, regex=False).sum()
    hospital_fraud_counts[hospital] = count

df_training['hospital_risk_score'] = df_training['provider_cleaned'].map(hospital_fraud_counts).fillna(0)

print("\nMedical & Benefit Features:")
print(df_training[['ailments', 'ailment_type_encoded', 'benefit', 'benefit_type_encoded', 
                   'broad_benefits_encoded', 'diagnosis_group', 'diagnosis_group_encoded', 
                   'provider', 'hospital_risk_score']].head())

In [ ]:
# First, let's examine the exact column names in your DataFrames
print("Training Dataset Columns:")
print(df_training.columns.tolist())

print("\nFraud Claims 1 Columns:")
print(df_fraud_claims1.columns.tolist())

print("\nFraud Claims 2 Columns:")
print(df_fraud_claims2.columns.tolist())

In [ ]:
# Cell 18: Feature Engineering: Behavioral & Derived Features
# is_first_claim for CLAIMANT or MAIN MEMBER
df_training['is_first_claim_claimant'] = df_training.groupby('claimant_suddo')['ailment_date'].rank(method='first') == 1
df_training['is_first_claim_main_member'] = df_training.groupby('member_suddo')['ailment_date'].rank(method='first') == 1

# claim_pattern_anomaly_score (Isolation Forest)
numerical_features_for_if = [
    'total_payable', 'cover_limit', 'days_since_last_visit',
    'claim_amount_mean_by_claimant', 'claim_amount_dev_by_claimant',
    'average_claim_per_broad_benefit', 'deviation_claim_per_broad_benefit',
    'service_charge_mean_by_ailment', 'service_charge_dev_by_ailment',
    'provider_service_mean_charge', 'provider_service_dev_charge',
    'repeat_claimant_count', 'claim_ratio', 'average_claim_per_user',
    'claim_amount_variance_claimant', 'claim_amount_variance_company',
    'company_total_claims', 'age', 'hospital_risk_score',
    'is_inpatient_benefit', 'is_outpatient_benefit', 'is_optical_benefit',
    'is_dental_benefit', 'is_maternity_benefit', 'is_last_expense_benefit',
    'age_rel_mismatch_flag', 'spouse_gender_mismatch_flag',
    'gender_diagnosis_mismatch_flag', 'age_diagnosis_mismatch_flag'
]

for col in numerical_features_for_if:
    if col in df_training.columns:
        df_training[col] = df_training[col].replace([np.inf, -np.inf], np.nan)
        df_training[col] = df_training[col].fillna(df_training[col].median())

iso_forest = IsolationForest(random_state=42, contamination=0.01)
features_for_if_present = [col for col in numerical_features_for_if if col in df_training.columns]
df_training['claim_pattern_anomaly_score'] = iso_forest.fit_predict(df_training[features_for_if_present])
df_training['claim_pattern_anomaly_score'] = df_training['claim_pattern_anomaly_score'].apply(lambda x: 1 if x == -1 else 0)

# company_claim_trend_zscore
df_training['provider_claim_zscore'] = df_training.groupby('provider')['total_payable'].transform(lambda x: (x - x.mean()) / x.std()).fillna(0)

# rel_frequency_score
df_training['rel_frequency_score'] = df_training.groupby('relationship')['member_suddo'].transform('count')

# is_high_risk_location
df_training['is_high_risk_location'] = df_training['provider_cleaned'].isin(all_fraud_hospitals).astype(int)

# company_fraud_incident_flag
# Using correct column names from your dataset:
# For df_fraud_claims1: 'company' column doesn't exist, using 'payee' instead
# For df_fraud_claims2: using 'reported_fraud_cases' instead of 'subject'
fraudulent_entities_from_fraud1 = df_fraud_claims1['payee'].dropna().astype(str).str.lower().unique()
fraudulent_entities_from_fraud2 = df_fraud_claims2['reported_fraud_cases'].dropna().astype(str).str.lower().unique()
all_fraudulent_companies = np.unique(np.concatenate([fraudulent_entities_from_fraud1, fraudulent_entities_from_fraud2]))

df_training['company_fraud_incident_flag'] = df_training['company'].astype(str).str.lower().isin(all_fraudulent_companies).astype(int)

print("\nBehavioral & Derived Features:")
print(df_training[[
    'claimant_suddo', 'is_first_claim_claimant', 'member_suddo', 'is_first_claim_main_member',
    'claim_pattern_anomaly_score', 'provider_claim_zscore', 'rel_frequency_score',
    'is_high_risk_location', 'company_fraud_incident_flag'
]].head())

In [ ]:

# Assuming df_training is your main DataFrame

# --- New Feature 1: Time since last claim for a similar ailment ---
# This requires sorting by claimant, ailment, and date.
df_training = df_training.sort_values(by=['claimant_suddo', 'ailments', 'ailment_date'])
df_training['days_since_last_ailment'] = df_training.groupby(['claimant_suddo', 'ailments'])['ailment_date'].diff().dt.days.fillna(0)

# --- New Feature 2: Time since last claim at the same provider ---
# This requires sorting by claimant, provider, and date.
df_training = df_training.sort_values(by=['claimant_suddo', 'provider', 'ailment_date'])
df_training['days_since_last_provider_visit'] = df_training.groupby(['claimant_suddo', 'provider'])['ailment_date'].diff().dt.days.fillna(0)

# --- New Feature 3: Day of the week and month ---
# These are simple extractions from the ailment date.
df_training['day_of_week'] = df_training['ailment_date'].dt.dayofweek
df_training['month_of_year'] = df_training['ailment_date'].dt.month

# --- New Feature 4: Provider Behavior Statistics ---
# Group by provider to calculate their average claim amount and claim count
provider_stats = df_training.groupby('provider')['total_payable'].agg(['mean', 'count']).reset_index()
provider_stats.rename(columns={'mean': 'provider_avg_claim', 'count': 'provider_claim_count'}, inplace=True)

# Merge these new features back into the main DataFrame
df_training = df_training.merge(provider_stats, on='provider', how='left')

print("New Features Added:")
print(df_training[['days_since_last_ailment', 'days_since_last_provider_visit', 
                   'day_of_week', 'month_of_year', 'provider_avg_claim', 'provider_claim_count']].head())

In [ ]:
# Cell 19: Final Feature Selection for Model Building
numerical_features = [
    'days_since_last_visit',
    'claim_amount_mean_by_claimant', 'claim_amount_dev_by_claimant', 'claim_amount_var_by_claimant',
    'average_claim_per_broad_benefit', 'deviation_claim_per_broad_benefit',
    'age', 'age_rel_mismatch_flag', 'spouse_gender_mismatch_flag',
    'service_charge_mean_by_ailment', 'service_charge_dev_by_ailment', 'service_charge_pct_dev_by_ailment',
    'gender_diagnosis_mismatch_flag', 'age_diagnosis_mismatch_flag',
    'provider_service_mean_charge', 'provider_service_dev_charge', 'provider_service_pct_dev_charge',
    'repeat_claimant_count', 'claim_ratio', 'high_claim_flag',
    'average_claim_per_user', 'claim_amount_variance_claimant', 'claim_amount_variance_company',
    'company_total_claims',
    'ailment_type_encoded', 'benefit_type_encoded', 'broad_benefits_encoded', 'diagnosis_group_encoded',
    'hospital_risk_score',
    'is_first_claim_claimant', 'is_first_claim_main_member', 'claim_pattern_anomaly_score',
    'provider_claim_zscore', 'rel_frequency_score', 'is_high_risk_location', 'company_fraud_incident_flag',
    'is_inpatient_benefit', 'is_outpatient_benefit', 'is_optical_benefit',
    'is_dental_benefit', 'is_maternity_benefit', 'is_last_expense_benefit'
]

categorical_columns_to_keep_original = [
    'gender(claimant)', 'relationship', 'company', 'broad_benefit', 
    'ailments', 'benefit', 'provider', 'diagnosis_group'
]

# Add main_member_gender if it exists (from your columns list it's 'main_member_gender')
if 'main_member_gender' in df_training.columns and df_training['main_member_gender'].dtype == 'object':
    categorical_columns_to_keep_original.append('main_member_gender')

# Filter to only include columns that exist in the dataframe
final_numerical_features = [col for col in numerical_features if col in df_training.columns]
final_features_for_model = list(set(final_numerical_features + categorical_columns_to_keep_original))

print("\nNumber of final features:", len(final_features_for_model))
print("\nSample of Final Features for Model:")
print(df_training[final_features_for_model].head())

In [ ]:
# First, let's add all required imports at the top
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, 
                           roc_auc_score, confusion_matrix)
import matplotlib.pyplot as plt

# Cell 20: Model Building - Complete Version
try:
    # Prepare the feature matrix X with one-hot encoding
    current_categorical_cols = [col for col in categorical_columns_to_keep_original if col in df_training.columns]
    X = pd.get_dummies(df_training[final_features_for_model], 
                      columns=current_categorical_cols, 
                      dummy_na=False,
                      drop_first=True)

    # Define the target variable (using the anomaly score we created)
    y = df_training['claim_pattern_anomaly_score']

    # Ensure target variable isn't in features
    if 'claim_pattern_anomaly_score' in X.columns:
        X = X.drop('claim_pattern_anomaly_score', axis=1)

    # Split data into training and testing sets (70% train, 30% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.3, 
        random_state=42, 
        stratify=y  # Maintain class balance
    )

    # Initialize and train a RandomForestClassifier with balanced class weights
    model = RandomForestClassifier(
        n_estimators=200,
        random_state=42, 
        class_weight='balanced',
        max_depth=10,
        min_samples_split=10
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability estimates

    # Evaluate the model
    print("\n--- Model Evaluation ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # Display feature importances
    feature_importances = pd.Series(
        model.feature_importances_, 
        index=X.columns
    ).sort_values(ascending=False)

    print("\nFeature Importances (Top 20):")
    print(feature_importances.head(20))

    # Additional evaluation metrics
    print("\nAdditional Metrics:")
    print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    # Plot feature importances
    plt.figure(figsize=(12, 8))
    feature_importances.head(20).plot(kind='barh')
    plt.title('Top 20 Important Features')
    plt.xlabel('Importance Score')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"An error occurred: {e}")
    print("Make sure all required columns exist in df_training")
    print("Current columns in df_training:", df_training.columns.tolist())

In [ ]:
# Display the first few rows of your dataset
df_training.head()


In [ ]:
import platform
print(platform.python_version())
print(platform.architecture())


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch for Autoencoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Load your data
df = df_training.copy()

# Define features to use
features = [
    'age(claimant)', 'days_since_last_visit', 'day_of_month_visited',
    'month_visited', 'hospital_risk_score', 'claim_pattern_anomaly_score',
    'provider_claim_zscore', 'rel_frequency_score'
]

X = df[features].copy().fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# === Step 1: Scale features with RobustScaler ===
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)  # X is your original feature matrix

# === Step 2: Train KMeans model ===
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_cluster = kmeans.fit_predict(X_scaled)

# === Step 3: Store cluster labels in DataFrame ===
df['kmeans_cluster'] = kmeans_cluster

# === Step 4: Calculate silhouette score ===
score = silhouette_score(X_scaled, kmeans_cluster)
print(f"Silhouette Score: {score:.4f}")


In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np

iso = IsolationForest(
    n_estimators=300,         # a bit more trees for stability
    contamination=0.05,       # target anomaly proportion (used for IF’s internal threshold)
    random_state=42,
    n_jobs=-1
).fit(X_scaled)

# Two scoring APIs:
# - decision_function: higher = more normal (less anomalous)
# - score_samples:     higher = more normal (less anomalous) but on a different scale
# We invert ONE of them so that higher = riskier
if_decision = iso.decision_function(X_scaled)         # higher = safer
if_component = -if_decision                           # invert -> higher = riskier (✅)

# Alternatively:
# if_component = -iso.score_samples(X_scaled)         # also higher = riskier (✅)

df['if_component'] = if_component.astype(float)


In [ ]:
import numpy as np

def rank01(a):
    a = np.asarray(a, dtype=float)
    a[np.isnan(a)] = np.nanmedian(a)
    r = a.argsort().argsort().astype(float)
    return r / max(len(a) - 1, 1)

parts = [
    rank01(df["if_component"].values),             # IF (inverted) -> higher = riskier
    rank01(df["autoencoder_anomaly_score"].values),# AE error      -> higher = riskier
    rank01(df["kmeans_min_distance"].values)       # KMeans dist   -> higher = riskier
]

df["combined_anomaly_score"] = np.mean(parts, axis=0)  # 0..1, higher = riskier


In [ ]:
# Distance to the closest cluster center (higher = farther = riskier)
kmeans_min_dist = kmeans.transform(X_scaled).min(axis=1)
df['kmeans_min_distance'] = kmeans_min_dist.astype(float)


In [ ]:
# Convert to torch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
dataset = TensorDataset(X_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = Autoencoder(input_dim=X_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    total_loss = 0
    for batch in loader:
        data = batch[0]
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/20, Loss: {total_loss:.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    recon = model(X_tensor)  # shape: [n_samples, n_features]
    # MSE per row (you can also try MAE which is more robust to outliers)
   mae = torch.mean(torch.abs(recon - X_tensor), dim=1).cpu().numpy()
df["autoencoder_anomaly_score"] = mae  # higher = more anomalous ✅
print(df["autoencoder_anomaly_score"].describe())

In [ ]:
import numpy as np
import pandas as pd

def rank01(a: np.ndarray) -> np.ndarray:
    """Ranks to [0,1]; robust to scale/outliers/NaNs."""
    a = np.asarray(a, dtype=float)
    if np.isnan(a).any():
        a[np.isnan(a)] = np.nanmedian(a)
    r = a.argsort().argsort().astype(float)
    return r / max(len(a) - 1, 1)

parts = []

# --- Isolation Forest component ---
# Expect a CONTINUOUS score with higher = riskier.
# If you previously used fit_predict (±1 labels), this will catch it.
if 'if_component' in df.columns:
    if_vec = df['if_component'].astype(float).values
else:
    # fall back to your existing column name
    if 'iso_anomaly_score' not in df.columns:
        raise ValueError("Need either df['if_component'] (preferred) or df['iso_anomaly_score'].")
    raw = df['iso_anomaly_score'].astype(float).values

    # guard: if it's just labels (-1/+1), stop and fix upstream
    uniq = pd.Series(np.unique(raw)).sort_values().tolist()
    if set(uniq).issubset({-1.0, 1.0}):
        raise ValueError(
            "iso_anomaly_score looks like labels (-1/+1). "
            "Compute a continuous score instead, e.g.: "
            "df['if_component'] = -iso.decision_function(X_scaled)"
        )
    # If raw is already oriented as higher=riskier, keep it.
    # If it's the decision_function (higher=safer), invert it upstream instead of guessing here.
    if_vec = raw

parts.append(rank01(if_vec))

# --- Autoencoder reconstruction error (higher = riskier) ---
if 'autoencoder_anomaly_score' in df.columns:
    parts.append(rank01(df['autoencoder_anomaly_score'].astype(float).values))
else:
    print("[WARN] 'autoencoder_anomaly_score' not found; combining without AE.")

# --- Optional: KMeans distance to nearest centroid (higher = farther = riskier) ---
if 'kmeans_min_distance' in df.columns:
    parts.append(rank01(df['kmeans_min_distance'].astype(float).values))

# --- Combine (simple average of ranks) ---
df['combined_anomaly_score'] = np.mean(parts, axis=0)  # 0..1, higher = riskier


In [ ]:
# ===== Time-aware validation slice + frozen threshold =====
import pandas as pd
import numpy as np
import os, json
from datetime import datetime, timezone

# --- Config ---
DATE_COL    = "ailment_date"   # your dataset has this column
REVIEW_RATE = 0.10             # e.g., target 10% review capacity
VERSION     = "v1"             # bump when you recalibrate

# --- Parse dates robustly (Kenya-style dd/mm/yyyy and also '2/6/2024') ---
df = df.copy()
# If column is numeric Excel serials, handle that; else assume strings with day-first
if np.issubdtype(df[DATE_COL].dtype, np.number):
    parsed = pd.to_datetime(df[DATE_COL], errors="coerce", origin="1899-12-30", unit="D")
else:
    parsed = pd.to_datetime(df[DATE_COL], errors="coerce", dayfirst=True)

df["_dt"] = parsed
df = df.dropna(subset=["_dt"]).sort_values("_dt")

# --- Use most recent ~20% as validation (you can also set a calendar cutoff if preferred) ---
cutoff = df["_dt"].quantile(0.80)   # first 80% "train-ish", last 20% "validation-ish"
df_valid = df[df["_dt"] >= cutoff].copy()
print(f"Validation window: {df_valid['_dt'].min().date()} → {df_valid['_dt'].max().date()} | n={len(df_valid):,}")

# --- Combine components via rank-averaging (robust, scale-free) ---
def rank01(a):
    a = np.asarray(a, dtype=float)
    a[np.isnan(a)] = np.nanmedian(a)
    r = a.argsort().argsort().astype(float)
    return r / max(len(a) - 1, 1)

parts = []
need_cols = []

if 'if_component' in df_valid.columns:
    parts.append(rank01(df_valid['if_component'].values))
else:
    need_cols.append('if_component')

if 'autoencoder_anomaly_score' in df_valid.columns:
    parts.append(rank01(df_valid['autoencoder_anomaly_score'].values))

if 'kmeans_min_distance' in df_valid.columns:
    parts.append(rank01(df_valid['kmeans_min_distance'].values))

if not parts:
    raise ValueError(f"No anomaly components found on df_valid. Missing: {need_cols}")

df_valid['combined_anomaly_score'] = np.mean(parts, axis=0)  # 0..1, higher = riskier
print(df_valid['combined_anomaly_score'].describe(percentiles=[.5, .9, .95, .98]))


In [ ]:
# --- Freeze threshold at capacity target ---
thr = float(np.quantile(df_valid['combined_anomaly_score'], 1 - REVIEW_RATE))

tuning = {
    "version": VERSION,
    "created_at": datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
    "review_rate_target": float(REVIEW_RATE),
    "threshold_combo_rank": thr,
    "validation_window": {
        "start": df_valid["_dt"].min().strftime("%Y-%m-%d"),
        "end":   df_valid["_dt"].max().strftime("%Y-%m-%d"),
        "n": int(len(df_valid))
    },
    # Optional: precompute a "High" band (top 2%) if you want a 3-band UI
    "band_high_cut": float(np.quantile(df_valid['combined_anomaly_score'], 0.98))
}

os.makedirs("models", exist_ok=True)
with open(f"models/tuning_{VERSION}.json", "w") as f:
    json.dump(tuning, f, indent=2)

print(f"[OK] Saved frozen threshold to models/tuning_{VERSION}.json")
print(f"[INFO] Threshold @ {int(REVIEW_RATE*100)}% review = {thr:.6f}")


In [ ]:
# ===== 3) Freeze the threshold at your capacity target and save to JSON =====
os.makedirs("models", exist_ok=True)

thr = float(np.quantile(df_valid['combined_anomaly_score'], 1 - REVIEW_RATE))

tuning = {
    "version": VERSION,
    "created_at": datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
    "review_rate_target": float(REVIEW_RATE),
    "threshold_combo_rank": thr,
    "validation_window": {
        "start": df_valid[DATE_COL].min().strftime("%Y-%m-%d"),
        "end":   df_valid[DATE_COL].max().strftime("%Y-%m-%d"),
        "n": int(len(df_valid))
    },
    # Optional: precompute band cuts for the UI (e.g., top 2% = High)
    "band_high_cut": float(np.quantile(df_valid['combined_anomaly_score'], 0.98))
}

with open(f"models/tuning_{VERSION}.json", "w") as f:
    json.dump(tuning, f, indent=2)

print(f"[OK] Saved frozen threshold to models/tuning_{VERSION}.json")
print(f"[INFO] Threshold @ {int(REVIEW_RATE*100)}% review = {thr:.6f}")


In [ ]:
importances = clf.feature_importances_
sns.barplot(x=importances, y=X_full.columns)
plt.title("Top Feature Importances")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df[['combined_anomaly_score', 'fraud_label']].to_csv("anomaly_detection_report.csv", index=False)
print("✅ Report saved as 'anomaly_detection_report.csv'")


In [ ]:
# Threshold to define fraud: top 3% of most anomalous
threshold = df['combined_anomaly_score'].quantile(0.97)
df['fraud_label'] = (df['combined_anomaly_score'] > threshold).astype(int)

# Optional: View distribution
print("Fraud label distribution:")
print(df['fraud_label'].value_counts())



In [ ]:
# Filter fraud-labeled claims (where fraud_label == 1)
fraud_claims = df[df['fraud_label'] == 1]

# Save to CSV
fraud_claims.to_csv("suspic_claims2.csv", index=False)
print("✅ Saved flagged fraud claims to 'suspic_claims2.csv'")


In [ ]:
# Make predictions on the full dataset (or remaining dataset)
y_full_pred = clf.predict(X_full)

# Count the number of predicted fraudulent claims
total_predicted_fraud = sum(y_full_pred)
print(f"Total predicted suspicious fraudulent claims: {total_predicted_fraud}")

# If you want to see the distribution
print("\nPredicted fraud label distribution:")
print(pd.Series(y_full_pred).value_counts())

# Optional: Save the predictions back to the dataframe
df['predicted_fraud_label'] = y_full_pred

# If you want to save the predicted fraudulent claims to a new file
predicted_fraud_claims = df[df['predicted_fraud_label'] == 1]
predicted_fraud_claims.to_csv("predicted_sus_claims.csv", index=False)
print("✅ Saved predicted fraudulent claims to 'predicted_sus_claims.csv'")

In [ ]:
# === EXPORT FITTED OBJECTS (tailored to your notebook) ===
# Prereqs in your notebook:
# - scaler: RobustScaler fitted on X
# - kmeans: KMeans fitted on X_scaled
# - iso: IsolationForest fitted on X_scaled
# - features: list of feature names used to build X in the SAME ORDER
# - df: DataFrame where df['combined_anomaly_score'] exists (from iso + autoencoder)
# - model: trained PyTorch autoencoder (optional export below)

import os, json, joblib, numpy as np
from datetime import datetime

# 1) Create models/ folder
os.makedirs("models", exist_ok=True)

# 2) Persist core sklearn artifacts and metadata
artifacts = {
    "scaler": scaler,                 # RobustScaler you fitted
    "kmeans": kmeans,                 # KMeans you fitted
    "iforest": iso,                   # IsolationForest you fitted
    "feature_names": list(features),  # EXACT order used during training
    "feature_version": "f1.0",
    "feature_medians": feature_medians,
}

joblib.dump(artifacts, "models/anomaly_v1.joblib")

# 3) (Optional) Persist the autoencoder for future use
#    We save two things:
#    - the model weights (state_dict)
#    - lightweight metadata to help reconstruct the AE at inference
#    If you don't plan to use AE at runtime yet, you can skip this block safely.
try:
    import torch
    ae_meta = {
        "input_dim": int(X_scaled.shape[1]),
        "arch": "fc-32-16-encoder",   # free-text note: adjust if you change architecture
    }
    torch.save({
        "state_dict": model.state_dict(),
        "meta": ae_meta
    }, "models/autoencoder_v1.pt")
    print("[INFO] Saved models/autoencoder_v1.pt (PyTorch).")
except Exception as _e:
    print("[WARN] Autoencoder export skipped (no torch/model?) ->", _e)

# 4) Freeze a GLOBAL threshold from your combined score (capacity-driven)
#    Your combined score is already in df['combined_anomaly_score'].
#    Pick a review capacity target (e.g., top 10% flagged).
import numpy as np

review_rate = 0.10   # <<< change this to match ops capacity (e.g., 0.08 .. 0.12)
if "combined_anomaly_score" not in df.columns:
    raise ValueError("df['combined_anomaly_score'] not found. Make sure you computed it earlier.")

combo_valid = df["combined_anomaly_score"].astype(float).to_numpy()
thr = float(np.quantile(combo_valid, 1 - review_rate))

tuning = {
    "version": "v1",
    "created_at": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "review_rate_target": float(review_rate),
    "threshold_combo_rank": thr,
    "notes": "Threshold frozen from current combined_anomaly_score distribution.",
    # (Optional) keep handy extra cuts for bands if you want in the UI:
    "band_high_cut": float(np.quantile(combo_valid, 0.98)),
}

with open("models/tuning_v1.json", "w") as f:
    json.dump(tuning, f, indent=2)

print("Saved: models/anomaly_v1.joblib and models/tuning_v1.json")
print(f"[INFO] Frozen threshold @ target {review_rate:.0%} -> {thr:.6f}")

# ... your existing export code above ...

print("Saved: models/anomaly_v1.joblib and models/tuning_v1.json")
print(f"[INFO] Frozen threshold @ target {review_rate:.0%} -> {thr:.6f}")

# === OPTIONAL: copy artifacts to your Streamlit app's models/ folder ===
from pathlib import Path
import shutil

# where the notebook wrote them (current notebook working dir)
src_dir = Path.cwd() / "models"

# where your Streamlit app expects them (adjust to your real app path)
dst_dir = Path("project/models")  # e.g., repo_root/project/models
dst_dir.mkdir(parents=True, exist_ok=True)

for name in ["anomaly_v1.joblib", "tuning_v1.json", "autoencoder_v1.pt"]:
    src = src_dir / name
    if src.exists():
        shutil.copy2(src, dst_dir / name)
        print(f"Copied {src} -> {dst_dir/name}")
    else:
        print(f"[WARN] Not found: {src} (skipped)")



In [ ]:
import joblib, json
art = joblib.load("models/anomaly_v1.joblib")
print(art.keys())        # expect: scaler, kmeans, iforest, feature_names, ...
print(len(art["feature_names"]))
with open("models/tuning_v1.json") as f:
    print(json.load(f))


In [ ]:
print(df_training.columns)
